In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from random import randint

In [12]:
df_cards=pd.read_csv('cartas_truco.csv')

In [13]:
df_cards

,id,id_palo,palo,carta,ranking,P_derrotar,P_obtener
0,0,1,espada,1,1,1.00,0.075
1,1,1,espada,2,6,0.72,0.277
2,2,1,espada,3,5,0.82,0.277
3,3,1,espada,4,14,0.00,0.277
4,4,1,espada,5,13,0.10,0.277
5,5,1,espada,6,12,0.21,0.277
6,6,1,espada,7,3,0.95,0.075
7,7,1,espada,10,10,0.36,0.277
8,8,1,espada,11,9,0.46,0.277
9,9,1,espada,12,8,0.56,0.277


In [156]:
def draw_cards(cards,deck):
    hay_envido=False
    cartas_envido=[]
    envido=0
    deck=list(deck)
    
    card_1 = deck[randint(0, len(deck)-1)]
    deck.remove(card_1)
    envido=cards["carta"].iloc[card_1-1]
    
    card_2 = deck[randint(0, len(deck)-1)]
    if cards["id_palo"].iloc[card_1-1] == cards["id_palo"].iloc[card_2-1]:
        hay_envido=True
        cartas_envido.append(card_1)
        cartas_envido.append(card_2)
    if cards["carta"].iloc[card_2-1] > envido: envido=cards["carta"].iloc[card_2-1]
    deck.remove(card_2)
    
    card_3 = deck[randint(0, len(deck)-1)]
    if cards["carta"].iloc[card_3-1] > envido: envido=cards["carta"].iloc[card_3-1]
    if cards["id_palo"].iloc[card_3-1] == cards["id_palo"].iloc[card_1-1]:
        hay_envido=True
        cartas_envido.append(card_3)
        cartas_envido.append(card_1)
    if cards["id_palo"].iloc[card_3-1] == cards["id_palo"].iloc[card_2-1]: 
        hay_envido=True
        cartas_envido.append(card_3)
        cartas_envido.append(card_2)
    deck.remove(card_3)
    
    if hay_envido:
        pre_envido=cards["carta"].iloc[cartas_envido[0]]+cards["carta"].iloc[cartas_envido[1]]
        if pre_envido >= 20: envido=20
        elif cards["carta"].iloc[cartas_envido[0]]>9 or cards["carta"].iloc[cartas_envido[1]]>9:
            envido =pre_envido +10
        else: envido =pre_envido +20
        
    return deck, card_1,card_2,card_3,envido,hay_envido

In [91]:
def call_envido(envido_1,envido_2,pie):
    print("El jugador 1 declara "+str(envido_1)+" puntos")
    print("El jugador 2 declara "+str(envido_2)+" puntos")
    if envido_1>envido_2:
        print("Ha ganado el envido el jugador 1")
        return 1 #jugador 1, 2 puntos
    elif envido_1 <envido_2:
        print("Ha ganado el envido el jugador 2")
        return 2
    else: #si ambos son iguales
        print("Empatan, gana el mano")
        return 0 #representa al jugador 1 o 2, al que sea el mano

In [92]:
def call_truco():
    return True

In [93]:
def fijarese_fin_por_puntaje(puntaje):
    if puntaje[0]>=15 or puntaje[1]>=15: 
        return 1
        declarar_ganador(puntaje)
    else: return 0

In [94]:
def declarar_ganador(puntaje):
    ganador=0
    if puntaje[0] > puntaje[1]: 
        ganador=1
    else:
        ganador=2
        print("el ganador es el jugador "+str(ganador))
        return ganador

In [97]:
def cards_clash(cards,card1,card2):
    if cards["ranking"].iloc[card1]>cards["ranking"].iloc[card2]: return 1
    elif cards["ranking"].iloc[card1]<cards["ranking"].iloc[card2]: return 2
    else: return 0

In [139]:
def jugar_una_mano(df_cards,points):
    #vars
    players=[1,2]
    points_f=[0,0]#puntaje final jugadores 1 y 2
    
    #primera mano
    result_draw=draw_cards(df_cards,df_cards.index)
    remainer_deck= result_draw[0]
    hand_1=[result_draw[1],result_draw[2],result_draw[3]]
    envido_hand_1=[result_draw[4],result_draw[5]]
    result_draw=draw_cards(df_cards,remainer_deck)
    hand_2=[result_draw[1],result_draw[2],result_draw[3]]
    envido_hand_2=[result_draw[4],result_draw[5]]
    
    #flags
    envido_declarado=False
    truco_declarado=False
    se_empardo=False
    see_p2_hand=True
    truco_rechazado=False
    
    pie=randint(players[0], players[-1]) #definir quien va primero
    print("el pie es el jugador "+str(pie))
    
    first_result=0
    second_result=0
    third_result=0
    final_result=0
    played_cards_p1=[]
    played_cards_p2=[]
    #global #all_results
    
    #random play

    #show hand
    print("su mano es:")
    print(df_cards.iloc[hand_1])
    if see_p2_hand:
        print("mano jugador 2:")
        print(df_cards.iloc[hand_2])
    
    next_player=pie
    #first round
    result=round_x(1,next_player,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,df_cards,pie)
    first_result=result[0]
    remainer_hand_1=result[1]
    remainer_hand_2=result[2]
    points_f[0]=result[3][0]
    points_f[1]=result[3][1]
    envido_declarado=result[4]
    truco_declarado=result[5]
    if first_result==0:
        se_empardo=True
    if truco_rechazado==True:
        #all_results.append(result)
        return points_f
    
    else: #second round
        
        #all_results.append(result)
        if pie==1: next_player==2
        else: next_player==1
        
        result=round_x(2,next_player,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,df_cards,pie)
        second_result=result[0]
        remainer_hand_1=result[1]
        remainer_hand_2=result[2]
        points_f[0]=result[3][0]
        points_f[1]=result[3][1]
        envido_declarado=result[4]
        truco_declarado=result[5]
        
        if first_result==second_result:
            print("El ganador de esta mano es el jugador "+str(second_result))
            points_f[0]+=points[0]
            points_f[1]+=points_f[0]
            if truco_declarado:
                if first_result ==1: points_f[0]+=2
                else: points_f[1]+=2
            #all_results.append(result)
            return points_f
        
        elif se_empardo:
            if first_result == second_result or second_result==0: final_result = first_result
            else: final_result = second_result
            print("El ganador de esta mano es el jugador "+str(final_result))
            points_f[0]+=points[0]
            points_f[1]+=points_f[0]
            if truco_declarado:
                if first_result ==1: points_f[0]+=2
                else: points_f[1]+=2
            #all_results.append(result)
            return points_f
                    
        else: #third round
            
            if next_player==1: next_player==2
            else: next_player==1
            
            result=round_x(2,next_player,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,df_cards,pie)
            third_result=result[0]
            remainer_hand_1=result[1]
            remainer_hand_2=result[2]
            points_f[0]=result[3][0]
            points_f[1]=result[3][1]
            envido_declarado=result[4]
            truco_declarado=result[5]
            
        if first_result == second_result or first_result == third_result: #caso 1: 1 jugador ganó 2/3 o 3/3
            final_result = first_result
        elif second_result==0 or third_result==0: #caso 2: un jugador ganó primera y empardó 2da o 3ra
            final_result = first_result
        elif second_result == third_result: #caso 3: un jugador perdió primera pero ganó 2/3
            final_result = second_result
        
        if final_result==1: ganador=1
        else: ganador = 2
        print("El ganador de esta mano es el jugador "+str(ganador))
        points_f[0]+=points[0]
        points_f[1]+=points_f[0]
        if truco_declarado:
            if ganador ==1: points_f[0]+=2
            else: points_f[1]+=2
    #all_results.append(result) #4
    return points_f
    
        #return(round_result,remainer_hand_1,remainer_hand_2,points,envido_declarado,truco_declarado)
        

In [169]:
def round_x(round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,df_cards,pie):
    global all_results
    if first_to_play==2:#en round_1 sería el pie
        #envido p2
        if envido_declarado==False:
            if envido_hand_2[1]: 
                envido_declarado=True
                envido_result=envido_play(2,envido_hand_1,envido_hand_2,points,pie)
                points[0]+=envido_result[0]
                points[1]+=envido_result[1]
                
        #play_card
        choice=card_choice(df_cards, hand_2,0)
        if round_n==3:choice=0
        first_play=hand_2[choice]
        remainer_hand_2=hand_2
        remainer_hand_2.remove(hand_2[choice])
        print("Jugador 2 elige la carta "+str(df_cards["carta"].iloc[first_play])+" de "+df_cards["palo"].iloc[first_play])
        
        #envido p1
        if envido_declarado==False:
            print("p1 desea declarar envido? eliga 1 para aceptar, 2 para rechazar")
            #choice=int(input ())
            if envido_hand_1[1]: 
                envido_declarado=True
                envido_result=envido_play(1,envido_hand_1,envido_hand_2,points,pie)
                points[0]+=envido_result[0]
                points[1]+=envido_result[1]
            
        #declarar_truco p1
        if round_n>1 and truco_declarado==False:
            print("presione 1 para declarar TRUCO, o 2 para no hacerlo")
            #choice=int(input ())
            choice=truco_choice(df_cards,hand)
            if choice==1:
                print("P1 ha declarado truco")
                truco_declarado=True
                player_truco=1
                print("P2 elige 1 para aceptar o 2 para rechazar")
                choice=truco_choice(df_cards,hand)
                if choice==1:
                    print("QUIERO")
                    truco_declarado=True
                    player_truco=1
                else:
                    truco_rechazado=True
                    points[0]+=1
                    all_results.append([round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,1])
                    return (2,hand_1,hand_2,points,envido_declarado,truco_declarado)
                
        #play_card
        print("seleccione numeros del 1 al 3 para elegir una carta, sus cartas son: ")
        print(df_cards.iloc[hand_1])
        #choice=int(input ())-1
        choice=card_choice(df_cards, hand_2,first_play)
        if round_n==3:choice=0
        first_response=hand_1[choice]
        remainer_hand_1=hand_1
        remainer_hand_1.remove(hand_1[choice])
        
        result=cards_clash(df_cards,first_play,first_response)
        print("Ha ganado esta ronda el jugador "+str(result))
        
        round_result=cards_clash(df_cards,first_play,first_response)
        
        all_results.append([round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,round_result])
        return(round_result,remainer_hand_1,remainer_hand_2,points,envido_declarado,truco_declarado)
    
    else: #s
        if envido_declarado==False:
            print("p1 desea declarar envido? eliga 1 para aceptar, 2 para rechazar")
            #choice=int(input ())
            if envido_hand_2[1]: 
                envido_declarado=True
                envido_result=envido_play(1,envido_hand_1,envido_hand_2,points,pie)
                points[0]+=envido_result[0]
                points[1]+=envido_result[1]
                
            
        print("seleccione numeros del 1 al 3 para elegir una carta, sus cartas son: ")
        choice=card_choice(df_cards, hand_1,0)
        #choice=int(input ())-1
        if round_n==3:choice=0
        first_play=hand_1[choice]
        remainer_hand_1=hand_1
        remainer_hand_1.remove(hand_1[choice])

        if envido_declarado==False:
            if envido_hand_2[1]: 
                envido_declarado=True
                envido_result=envido_play(2,envido_hand_1,envido_hand_2,points,pie)
                points[0]+=envido_result[0]
                points[1]+=envido_result[1]

        
        #declarar_truco p2
        if round_n>1 and truco_declarado==False:
            print("p1 presione 1 para declarar TRUCO, o 0 para no hacerlo")
            #choice=int(input ())
            choice=truco_choice(df_cards,hand)
            if choice==1:
                print("Ha declarado truco")
                truco_declarado=True
                player_truco=2
            else:
                truco_rechazado=True
                points[1]+=1
                all_results.append([round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,2])
                return (2,hand_1,hand_2,points,envido_declarado,truco_declarado)
            
        choice=card_choice(df_cards, hand_2,first_play)
        if round_n==3:choice=0
        first_response=hand_2[choice]
        remainer_hand_2=hand_2
        remainer_hand_2.remove(hand_2[choice])
        print("Jugador 2 elige la carta "+str(df_cards["carta"].iloc[first_play])+" de "+df_cards["palo"].iloc[first_play])
        
        if envido_declarado==False:
            print("p2 desea declarar envido? eliga 1 para aceptar, 2 para rechazar")
            #choice=int(input ())
            if envido_hand_2[1]: 
                    envido_declarado=True
                    envido_result=envido_play(1,envido_hand_1,envido_hand_2,points,pie)
                    points[0]+envido_result[0]
                    points[1]+envido_result[1]
                    
        result=cards_clash(df_cards,first_play,first_response)
        print("Ha ganado esta ronda el jugador "+str(result))
        
        round_result=cards_clash(df_cards,first_play,first_response)

        all_results.append([round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,round_result])
        return(round_result,remainer_hand_1,remainer_hand_2,points,envido_declarado,truco_declarado)

In [165]:
def envido_play(who_declared,envido_hand_1,envido_hand_2,points,pie):
    mano=0
    if pie==2: mano = 1
    else: mano=2
    if who_declared==2:
        print("Jugador 2 declara envido, eliga 1 para aceptar, 2 para rechazar, 3 para declarar envido")
        #choice=input ()
        choice=envido_choice(2,pie,envido_hand_2)
        if choice==1: 
            print("QUIERO")
            envido_result=call_envido(envido_hand_1,envido_hand_2,pie)
            if envido_result ==1: points[0]+=2
            elif envido_result ==2: points[1]+=2
            else: #empataron
                envido_result= mano
                if envido_result ==1: points[0]+=2
                elif envido_result ==2: points[1]+=2
        elif choice==2: 
            print("NO QUIERO")
            points[1] += 1
        else: 
            print("envido")
            envido_result=call_envido(envido_hand_1,envido_hand_2,pie)
            if envido_result ==1: points[0]+=4
            elif envido_result ==2: points[1]+=4
            else: #empataron
                envido_result= mano
                if envido_result ==1: points[0]+=4
                elif envido_result ==2: points[1]+=4
    else: 
        print("Jugador 1 declara envido, eliga 1 para aceptar, 2 para rechazar, 3 para declarar envido")
        choice_p2=envido_choice(2,pie,envido_hand_2)
        print(choice_p2)
        if choice_p2==1: 
            print("QUIERO")
            envido_result=call_envido(envido_hand_1,envido_hand_2,pie)
            if envido_result ==1: points[0]+=2
            elif envido_result ==2: points[1]+=2
            else: #empataron
                envido_result= mano
                if envido_result ==1: points[0]+=2
                elif envido_result ==2: points[1]+=2
        elif choice_p2==2: 
            print("NO QUIERO")
            points[0] += 1
        else: 
            print("envido")
            envido_result=call_envido(envido_hand_1,envido_hand_2,pie)
            if envido_result ==1: points[0]+=4
            elif envido_result ==2: points[1]+=4
            else: #empataron
                envido_result= mano
                if envido_result ==1: points[0]+=4
                elif envido_result ==2: points[1]+=4
    return points

In [101]:
def partida(df_cards):
    points=[0,0] #puntaje de jugadores 1 y 2
    while points[0]<15 and points[1]<15:
        points=jugar_una_mano(df_cards,points)
        fijarese_fin_por_puntaje(points)
        print("puntos del jugador 1: "+str(points[0]))
        print("puntos del jugador 2: "+str(points[1]))
    if points[0]>points[0]:
        print("el GANADOR es el jugador 1")
    elif points[0]<points[1]:
        print("el GANADOR es el jugador 2")
    else:
        print("Empate")

In [175]:
partida(df_cards)

el pie es el jugador 1
su mano es:
    id_palo    palo  carta  ranking  P_derrotar  P_obtener
id                                                        
13        2   basto      3        5        0.82      0.277
18        2   basto     10       10        0.36      0.277
3         1  espada      3        5        0.82      0.277
mano jugador 2:
    id_palo    palo  carta  ranking  P_derrotar  P_obtener
id                                                        
10        1  espada     12        8        0.56      0.277
4         1  espada      4       14        0.00      0.277
30        3    copa     12        8        0.56      0.277
p1 desea declarar envido? eliga 1 para aceptar, 2 para rechazar
Jugador 1 declara envido, eliga 1 para aceptar, 2 para rechazar, 3 para declarar envido
1
QUIERO
El jugador 1 declara [23, True] puntos
El jugador 2 declara [26, True] puntos
Ha ganado el envido el jugador 2
seleccione numeros del 1 al 3 para elegir una carta, sus cartas son: 


IndexError: list index out of range

In [146]:
#all_results=(n_ronda,hand1,hand2,envido_hand1,envido_hand2,played_card_p1,played_card_p2,pie,called_truco,called_envido,points_p1,points_p2)
all_results=[]

In [166]:
def envido_choice(player,pie,envido_hand):
    global df_envido
    mano=0
    soy_mano=False
    if pie!=player: soy_mano =True
    else: 
        if pie==1: mano=2
        else: mano=1 
    if envido_hand[1]: #Si tiene envido se fija las probabilidades de ganar con las cartas que tiene
        p_ganar=df_envido[(df_envido["Envido_low"]<=envido_hand[0]) & (df_envido["Envido_high"]>=envido_hand[0]) & (df_envido["Mano"]==soy_mano)]["P_ganar"].iloc[0]
        if p_ganar>=0.75: return 3
        elif p_ganar>=0.5: return 1
        else: return 2
    else: return 2 #si NO tiene envido rechaza

In [115]:
def truco_choice(df_cards,hand):
    total_p_derrotar=0.0
    for card in hand:
        total_p_derrotar+=df_cards["P_derrotar"].iloc[hand[card]]
    if total_p_derrotar > 150/len(hand): #si hay más de 50% de probabilidades de derrotar las cartas del oponente
        return 1
    else: return 2


In [116]:
df_envido = pd.read_csv('embido_truco.csv')

In [125]:
df_envido[(df_embido["Embido_low"]>=33) & (df_embido["Embido_low"]<=33) & (df_embido["Mano"]==True)]["P_ganar"].iloc[0]

1.0

In [174]:
def card_choice(df_cards, hand,first_card):
    best_card=0
    worst_card=1
    if first_card==0: #si es la primera carta de la ronda
        for card in hand:
            if df_cards["P_derrotar"].iloc[best_card]<df_cards["P_derrotar"].iloc[hand[card]]: best_card=card
        return best_card
    else: #si ya se jugó otra carta esta ronda
        for card in hand:
            if df_cards["P_derrotar"].iloc[first_card]<df_cards["P_derrotar"].iloc[hand[card]]: best_card=card
            else:
                for card in hand:
                    if df_cards["P_derrotar"].iloc[worst_card]>df_cards["P_derrotar"].iloc[hand[card]]: 
                        worst_card=card
                return worst_card
        return best_card
    

In [127]:
envido_choice(1,1,2,[6,True],0)

2

In [159]:
df_games=pd.DataFrame(all_results,columns =['round_n','first_to_play','hand_1','envido_hand_1','hand_2','envido_hand_2','points','envido_declarado','truco_declarado','pie','remainer_hand_1','remainer_hand_2','winner'])

In [160]:
#all_results.append([round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,round_result])

df_games

,round_n,first_to_play,hand_1,envido_hand_1,hand_2,envido_hand_2,points,envido_declarado,truco_declarado,pie,remainer_hand_1,remainer_hand_2,winner
0,1,1,[],"[28, True]",[],"[26, True]","[4, 0]",True,False,1.0,[],[],1.0
1,1,[],[],"[4, 0]",True,False,None,None,None,NaN,None,None,NaN
2,2,1,[],"[28, True]",[],"[26, True]","[4, 0]",True,True,1.0,[],[],0.0
3,2,1,[],"[28, True]",[],"[26, True]","[4, 0]",True,True,1.0,[],[],1.0
4,1,[],[],"[4, 0]",True,True,None,None,None,NaN,None,None,NaN
5,1,1,"[33, 26]","[30, True]","[2, 9]","[25, True]","[24, 18]",True,False,1.0,"[33, 26]","[2, 9]",1.0
6,1,"[33, 26]","[2, 9]","[24, 18]",True,False,None,None,None,NaN,None,None,NaN
7,2,"[33, 26]","[2, 9]","[24, 18]",True,False,None,None,None,NaN,None,None,NaN
8,1,1,"[5, 31]","[33, True]","[8, 19]","[12, False]","[1, 1]",True,False,1.0,"[5, 31]","[8, 19]",2.0
9,2,"[5, 31]","[8, 19]","[1, 1]",True,False,None,None,None,NaN,None,None,NaN


In [15]:
df_cards

,id,id_palo,palo,carta,ranking,P_derrotar,P_obtener
0,0,1,espada,1,1,1.00,0.075
1,1,1,espada,2,6,0.72,0.277
2,2,1,espada,3,5,0.82,0.277
3,3,1,espada,4,14,0.00,0.277
4,4,1,espada,5,13,0.10,0.277
5,5,1,espada,6,12,0.21,0.277
6,6,1,espada,7,3,0.95,0.075
7,7,1,espada,10,10,0.36,0.277
8,8,1,espada,11,9,0.46,0.277
9,9,1,espada,12,8,0.56,0.277


In [14]:
df_cards[(df_cards["id_palo"]==1) & (df_cards["carta"]==1)]["id"].iloc[0]
    

0